In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy as sc
from scipy import stats 
import scipy.stats as stat
import helpfunc as func
from helpfunc import Droplet_Values
from helpfunc import Radii_Charge_NotAveragedVel
from helpfunc import Radii_Charge_VelAVERAGED

# Skip down to Analysis 

In [19]:
# Plate Separation 
sigd = 0.0127E-3    # meters
sigx = 0.01E-3      # meters
sigt = 0.1          # fractional seconds

In [20]:
def Droplet_Values(n_drop, n_ionized, csv_name_as_string):
    # Funtion: Choose Droplet, Choose ionization, Call "file.csv"
    # Return a list of paramerters per droplet selected 
    # E.g.: droplet1_ionization2 = [velocities, Voltage]


    # Call the CSV file
    droplet = pd.read_csv(csv_name_as_string)

    # Select the specified droplet by droplet # and Ionization (rows)
    d1 = droplet[droplet['Droplet'].isin([n_drop]) & droplet['Ionized'].isin([n_ionized])]

    # Select the specified parameters (Velocities and Volts) (columns)
    d1 = d1[["Vf1","Vr1","Vf2","Vr2","Vf3","Vr3","vf av","vr av","Volts V","vf stdev", "vr stdev"]]

    # Convert to array, already is a float
    d1 = d1.to_numpy()

    # Assign a list 
    d1_values = []

    # use a for loop to unpack the 0th element into our desired list

    for value in d1[0]:
        d1_values.append(value)

    return d1_values

In [21]:
def CreateDropsDatabase(csv_file_name_as_string): # "filename.csv"
    # Function: Takes "file.csv" Returns list of lists for our drops
    # dependent on the function Droplet_Values()
    # E.g.: Drops = [d1[velocities,Voltage],...dn[velocities,Voltage]]
    # Full detial of function found in Oil-Drop.ipynb

    drops = []
   
    droplet_number = 1 
   
    while droplet_number is not None: 

        ionization_number = 0

       
        while ionization_number is not None: 
         

            try:
                d = Droplet_Values(droplet_number, ionization_number, csv_file_name_as_string)
        
                drops.append(d)
                ionization_number += 1

            except IndexError:
               
                if ionization_number == 0: 
                   
                    droplet_number = None
                    break
                else:
                    
                    ionization_number = None
        
        if droplet_number is not None:
            droplet_number += 1
    return drops 

# Pull all drops 

In [22]:
all_drops = CreateDropsDatabase("Velocities.csv")
print(f"\nWe have {len(all_drops)} drops to examine.")
print(f"Some have multiple ionizations, we will only examine 1 per set of ionizations.")
print(f"Each drop has {len(all_drops[0])} parameters.\n")

for x in all_drops: 
    print(x)


We have 40 drops to examine.
Some have multiple ionizations, we will only examine 1 per set of ionizations.
Each drop has 11 parameters.

[3.65e-05, 0.00025, 4.72e-05, 0.000263, 3.7e-05, 0.000132, 4.02e-05, 0.000215, 501.0, 6.01e-06, 7.25e-05]
[0.000119, 4.31e-05, 0.000116, 4.39e-05, 0.000114, 0.0002, 0.000116, 9.57e-05, 501.0, 2.71e-06, 9.04e-05]
[0.000125, 0.000179, 0.000114, 0.000208, 0.000125, 0.0002, 0.000121, 0.000196, 501.0, 6.56e-06, 1.54e-05]
[8.77e-05, 0.000556, 0.000125, 0.000625, 0.000114, 0.000556, 0.000109, 0.000579, 501.0, 1.91e-05, 4.01e-05]
[0.000111, 0.000714, 0.000119, 0.000833, 0.000122, 0.000833, 0.000117, 0.000794, 501.0, 5.61e-06, 6.87e-05]
[3.76e-05, 0.000185, 3.23e-05, 0.000217, 4.24e-05, 0.000172, 3.74e-05, 0.000192, 501.0, 5.06e-06, 2.32e-05]
[3.01e-05, 0.000417, 2.7e-05, 0.000333, 2.44e-05, 0.000357, 2.72e-05, 0.000369, 501.0, 2.87e-06, 4.29e-05]
[2.79e-05, 0.000357, 3.31e-05, 0.000263, 2.92e-05, 0.000313, 3.01e-05, 0.000311, 501.0, 2.69e-06, 4.7e-05]
[4.55

# Calculate the charge values

In [23]:
#Assign lists 
r_vel_notav, q_vel_notav = func.Radii_Charge_NotAveragedVel(all_drops)
# r_velav, q_velav = func.Radii_Charge_VelAVERAGED(all_drops)

In [24]:
#test pull paramater 
print(all_drops[-1][0])

6.41e-05


In [25]:
# # print to see list 
# print("Charges from velocites, not averaged velocites:")
# print(f"Amount of charge's = {len(q_vel_notav)}")
# print(q_vel_notav) # this values is defined later and it is a reshaping

In [26]:
def varq(ri, v_y0, v_yE, sigvf, sigvr):
    phi1 = ((18*np.pi)/501)
    insqrt = (n_o**3)/(2*g*p)
    phi2 = np.sqrt(insqrt)

    #radius input
    c = 1 + (b/(p*ri))
    incubrt = 1/c
    phi3 = incubrt**(3/2)

    phi = phi1*phi2*phi3

    #term1 sigma v_y0
    t1 = phi*d*(v_yE-3*v_y0)
    t2 = 2*np.sqrt(v_y0)

    term1 = ((t1/t2)**2)*sigvf**2

    #term2 sigma v_yE
    t3 = phi*d*np.sqrt(v_y0)

    term2 = (t3**2)*sigvr**2

    #term3 sigma d
    t4 = phi*np.sqrt(v_y0)*(v_yE-v_y0)

    term3 = (t4**2)*sigd**2

    #variance
    varq = term1+term2+term3 

    return varq



In [27]:
def sigmaCharge_NotAveragedVel(drops_list_of_lists): 
    # droplet = to loop through
    # drops a list of lists previously created

    # function takes list to loop through
    # returns two lists one for radius and one for charge 
    list_of_radius_values = []
    list_of_charge_values =[]


    for droplet in drops_list_of_lists: 
        # n = index value of 1droplets parameters
        # n = 0  
        # print(droplet)
        for i in range(3): 
            idx = i * 2

            # if n > 4: 
            #     break
            # else: 
            if droplet[idx] ==0 or droplet[idx+1] ==0: 
                list_of_radius_values.append(0)
                list_of_charge_values.append(0)
                print("no value vf.vr")
                continue
            else:
                # print(n)
                # print('new drop vf.vr')
                # print(droplet[n])
                # print(droplet[n+1])

                ri = radius(droplet[idx])
                list_of_radius_values.append(ri)

                #calculate charge 
                qi = q1(ri,droplet[idx],droplet[idx+1],droplet[8])
                list_of_charge_values.append(qi)
                # n += 2
    return list_of_radius_values, list_of_charge_values

In [28]:
print(sigd)

1.27e-05
